In [1]:
from data_generators import get_train_test_addition

#dimensjoner og størrelser til x og y
n_digits = 2
n_max = 3*n_digits
m = 10

#definerer størrelsen på parametermatrisene
d = 15
k = 5
p = 20

#henter treningsdata
data = get_train_test_addition(n_digits,samples_per_batch=100,n_batches_train=4)

In [2]:
from utils import onehot

x = data['x_train'][0]
X = onehot(x,m)
y = data['y_train'][0]

In [3]:
data['x_train'].shape

(4, 100, 6)

In [4]:
from layers import LinearLayer,EmbedPosition,Attention,Softmax,CrossEntropy,FeedForward

embed = EmbedPosition(n_max,m,d)
att1 = Attention(d,k)
ff1 = FeedForward(d,p)
un_embed = LinearLayer(d,m)
softmax = Softmax()
loss = CrossEntropy()

In [5]:
#vi ser at dLdW_2 tilhørende ff1 (først finner vi det lineære laget l2
#så nøkkel 'w' i params dict-en, så nøkkel 'd' for å finne den deriverte)
#nå er denne bare en nullmatrise 
ff1.l2.params['w']['d']

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.

In [6]:
#"manuelt" forward pass (tilsvarende algoritme 1)
z0 = embed.forward(X)
z1 = att1.forward(z0)
z2 = ff1.forward(z1)
z = un_embed.forward(z2)
Z = softmax.forward(z)

In [7]:
#evaluerer objektfunksjonen
L = loss.forward(Z,y)
print(L)

#finner den deriverte av objektfunksjonen mhp Z
dLdZ = loss.backward()

2.304645857015904


In [8]:
#"manuelt" backward pass (tilsvarende algoritme 2)
dLdz = softmax.backward(dLdZ)
dLdz2 = un_embed.backward(dLdz)
dLdz1 = ff1.backward(dLdz2)
dLdz0 = att1.backward(dLdz1)
embed.backward(dLdz0)

In [9]:
#etter backward pass har dLdW_2 fått verdier
ff1.l2.params['w']['d']

array([[ 7.11316356e-05,  1.90495525e-04, -4.53043131e-05,
         1.04102072e-04, -4.20189329e-05, -1.57720936e-05,
        -9.42090136e-05,  2.11193267e-05, -2.13131487e-04,
         9.53218341e-05,  6.31796020e-06, -3.30341854e-05,
        -6.48944261e-05,  6.93939535e-07,  8.99382560e-05,
        -9.96417970e-05, -1.05875476e-05,  9.92815656e-05,
         4.80592310e-05, -1.33901685e-04],
       [ 4.36927303e-04,  7.05000577e-04,  2.06093563e-04,
         1.71735628e-05,  5.06520510e-05,  3.63758179e-04,
         1.38025295e-04, -6.18198169e-05,  4.62118627e-04,
         7.70310270e-04, -1.03197190e-04,  1.55790681e-05,
         3.34174859e-05, -9.60224903e-06,  2.72935228e-04,
         3.95620957e-04,  2.19579983e-04,  2.99569717e-04,
         6.58022094e-04,  3.64209057e-04],
       [-4.19543271e-04, -5.94562079e-04,  1.24432226e-04,
         5.21030661e-07,  9.49472893e-05,  1.40615544e-04,
         3.98729384e-04, -9.90411940e-05,  4.11647503e-04,
        -3.14338740e-04, -4.2

In [10]:
from neural_network import NeuralNetwork

#vi kan samle lagene i en liste som vi bruker for å
#initialisere et nevralt nettverk der vi kan bruke forward() og backward() 
#for å oppnå det samme som vi gjorde manuelt over

layers = [embed,att1,ff1,un_embed,softmax]
nn = NeuralNetwork(layers)

In [11]:
#forward pass tilsvarende algoritme 1
Z = nn.forward(X)

#beregner loss med CrossEntropy
L = loss.forward(Z,y)
print(L)

#backward pass tilsvarende algoritme 2
dLdZ = loss.backward()
nn.backward(dLdZ)

2.304645857015904


In [12]:
#før optimering er W_2[0,0] gitt ved
#ff1 er det tredje laget i layers-listen (derav layers[2])
W_2_pre_opt = nn.layers[2].l2.params['w']['w'].copy()
print(W_2_pre_opt[0,0])

0.0005562762195827531


In [ ]:
import numpy as np

xs = data['x_train']
ys = data['y_train']

n_batches = xs.shape[0]
n_iters = 100
step_size = 0.1

#treningsløkke tilsvarende algoritme 4 (med gradient descent)
for j in range(n_iters):
    losses = []
    for i in range(n_batches):
        x = xs[i]
        y = ys[i]

        X = onehot(x,m)
        Z = nn.forward(X)

        losses.append(loss.forward(Z,y))
        dLdZ = loss.backward()
        nn.backward(dLdZ)
        nn.step_gd(step_size)
    mean_loss = np.mean(losses)
    print("Iterasjon ", str(j), " L = ",mean_loss, "")

In [14]:
import numpy as np

xs = data['x_train']
ys = data['y_train']

n_batches = xs.shape[0]
n_iters = 100
step_size = 0.01

#treningsløkke tilsvarende algoritme 4 (med gradient descent)
for j in range(n_iters):
    losses = []
    for i in range(n_batches):
        x = xs[i]
        y = ys[i]

        X = onehot(x,m)
        Z = nn.forward(X)

        losses.append(loss.forward(Z,y))
        dLdZ = loss.backward()
        nn.backward(dLdZ)
        nn.step_adam(step_size, j + 1)
    mean_loss = np.mean(losses)
    print("Iterasjon ", str(j), " L = ",mean_loss, "")

Iterasjon  0  L =  1.748032448205358 
Iterasjon  1  L =  1.7460115516598977 
Iterasjon  2  L =  1.743930165232553 
Iterasjon  3  L =  1.742361186610189 
Iterasjon  4  L =  1.7409170612123994 
Iterasjon  5  L =  1.7399883774571343 
Iterasjon  6  L =  1.7385921623667862 
Iterasjon  7  L =  1.7379723922021646 
Iterasjon  8  L =  1.7368855080395864 
Iterasjon  9  L =  1.735579187314603 
Iterasjon  10  L =  1.7347745922216367 
Iterasjon  11  L =  1.7337520567100932 
Iterasjon  12  L =  1.7326801685144435 
Iterasjon  13  L =  1.73190831917644 
Iterasjon  14  L =  1.7308272311849935 
Iterasjon  15  L =  1.7297996362033512 
Iterasjon  16  L =  1.729019599610047 
Iterasjon  17  L =  1.7279435634662186 
Iterasjon  18  L =  1.7268959539300512 
Iterasjon  19  L =  1.7262402402959804 
Iterasjon  20  L =  1.7250336492004068 
Iterasjon  21  L =  1.724276098854821 
Iterasjon  22  L =  1.7233860507467709 
Iterasjon  23  L =  1.7220511450697442 
Iterasjon  24  L =  1.7214492014402616 
Iterasjon  25  L =

In [ ]:
#etter trening ser vi at W_2 er gitt ved

W_2_post_opt = nn.layers[2].l2.params['w']['w'].copy()
print(W_2_post_opt[0,0])

#dersom differansen er større enn null har dette parameteret endret seg etter 
#gradient descent
print(W_2_post_opt[0,0] - W_2_pre_opt[0,0])


